# LangChain, PostgreSQL, PaLM API でドキュメント QA を行う例

以下のパッケージをインストールします。
- LangChain 関係のパッケージ
- PDF の扱いに必要なパッケージ
- CloudSQL の PostgreSQL インスタンスにアクセスするためのパッケージ

In [ ]:
!pip install --user \
  langchain==0.1.0 transformers==4.36.0 \
  pypdf==3.17.0 cryptography==41.0.6 \
  pg8000==1.30.2 cloud-sql-python-connector[pg8000]==1.4.2 \
  google-cloud-aiplatform==1.36.1

**注意：次のセルを実行する前にカーネルをリスタートしてください。**

テキストエンベディング API を試してみます。1つのテキストから、768次元の埋め込みベクトルが得られます。

In [1]:
from langchain_community.embeddings import VertexAIEmbeddings
embeddings = VertexAIEmbeddings(
    model_name='textembedding-gecko-multilingual@001')
embedding_vectors = embeddings.embed_documents(['今日は快晴です。'])

In [2]:
len(embedding_vectors), len(embedding_vectors[0])

(1, 768)

In [3]:
embedding_vectors[0][:5]

[0.0095707131549716,
 -0.02757399156689644,
 0.0025982840452343225,
 0.042188193649053574,
 -0.09588668495416641]

デジタル庁が一般公開している「[アジャイル開発実践ガイドブック](https://www.digital.go.jp/resources/standard_guidelines)」の PDF ファイルをダウンロードします。

In [4]:
base_url = 'https://raw.githubusercontent.com/google-cloud-japan/sa-ml-workshop/main'
!wget -q $base_url/genAI_book/PDF/agile-guidebook.pdf

PDF ファイルの内容をページごとに分割して、それぞれのページの埋め込みベクトルを生成します。

In [5]:
from langchain_community.document_loaders import PyPDFLoader
pages = PyPDFLoader('agile-guidebook.pdf').load()
page_contents = [page.page_content for page in pages]
embedding_vectors = embeddings.embed_documents(page_contents)

In [6]:
len(embedding_vectors), len(embedding_vectors[0])

(37, 768)

PostgreSQL に接続するためのコネクションプールを用意します。

In [7]:
import google.auth
import sqlalchemy
from google.cloud.sql.connector import Connector

_, project_id = google.auth.default()
region = 'asia-northeast1'
instance_name = 'genai-app-db'
INSTANCE_CONNECTION_NAME = '{}:{}:{}'.format(
    project_id, region, instance_name)
DB_USER = 'db-admin'
DB_PASS = 'genai-db-admin'
DB_NAME = 'docs_db'

connector = Connector()

def getconn():
    return connector.connect(
        INSTANCE_CONNECTION_NAME, 'pg8000',
        user=DB_USER, password=DB_PASS, db=DB_NAME)

pool = sqlalchemy.create_engine('postgresql+pg8000://', creator=getconn)

埋め込みベクトルをデータベースから削除、および、データベースに保存する関数を定義します。

In [8]:
def delete_doc(docid):
    with pool.connect() as db_conn:
        delete_stmt = sqlalchemy.text(
            'DELETE FROM docs_embeddings WHERE docid=:docid;'
        )
        parameters = {'docid': docid}
        db_conn.execute(delete_stmt, parameters=parameters)
        db_conn.commit()

def insert_doc(docid, uid, filename, page, content, embedding_vector):
    with pool.connect() as db_conn:
        insert_stmt = sqlalchemy.text(
            'INSERT INTO docs_embeddings \
             (docid, uid, filename, page, content, embedding) \
             VALUES (:docid, :uid, :filename, :page, :content, :embedding);'
        )
        parameters = {
            'docid': docid,
            'uid': uid,
            'filename': filename,
            'page': page,
            'content': content,
            'embedding': embedding_vector
        }
        db_conn.execute(insert_stmt, parameters=parameters)
        db_conn.commit()

先ほどのドキュメントの各ページの埋め込みベクトルをデータベースに保存します。

In [9]:
docid = 'dummy_id'
uid = 'dummy_uid'
filename = 'agile-guidebook.pdf'

delete_doc(docid)        
for c, embedding_vector in enumerate(embedding_vectors):
    page = c+1
    insert_doc(docid, uid, filename, page,
               page_contents[c], str(embedding_vector))

質問文から埋め込みベクトルを生成して、関連性の高いページ（埋め込みベクトルの値が近い）のトップ3を取得します。

In [10]:
question = 'アジャイル開発の採用に慎重になるべきケースはありますか？'
question_embedding = embeddings.embed_query(question)

with pool.connect() as db_conn:
    search_stmt = sqlalchemy.text(
        'SELECT filename, page, content, \
                1 - (embedding <=> :question) AS similarity \
         FROM docs_embeddings \
         WHERE uid=:uid \
         ORDER BY similarity DESC LIMIT 3;'
    )
    parameters = {'uid': uid, 'question': str(question_embedding)}
    results = db_conn.execute(search_stmt, parameters=parameters)

text = ''
source = []
for filename, page, content, _ in results:
    source.append({'filename': filename, 'page': page})
    text += content + '\n'

In [11]:
source

[{'filename': 'agile-guidebook.pdf', 'page': 16},
 {'filename': 'agile-guidebook.pdf', 'page': 17},
 {'filename': 'agile-guidebook.pdf', 'page': 12}]

得られたページのテキストに基づいて、質問の回答を生成します。

In [14]:
from langchain_community.llms import VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

llm = VertexAI(model_name='text-bison@002', 
               temperature=0.1, max_output_tokens=256)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=6000, chunk_overlap=200)
qa_chain = load_qa_chain(llm, chain_type='refine')
qa_document_chain = AnalyzeDocumentChain(
    combine_docs_chain=qa_chain, text_splitter=text_splitter)

prompt = '{} 日本語で3文程度にまとめて教えてください。'.format(question)
answer = qa_document_chain.invoke({'input_document': text, 'question': prompt})
print(answer['output_text'])

 アジャイル開発の採用に慎重になるべきケースは、大規模な情報システム、業務内容等が極めて複雑、あるいはミッションクリティカルなケースです。このような場合は、どこまでをあらかじめ詳細化するか、どの部分をアジャイルに開発するか、また、どのように品質を確保し、継続的に高めていくかといった判断が必要となります。
